# Ant versus Bee Classification using Pre-trained ResNet-18 Convolutional Neural Network Mode

### Objective : Practice using pre-trained neural networks to extract domain-specific features for new tasks

In [643]:

from __future__ import print_function, division          ##https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

# Read the pytorch tutorial to use a pre-trained “ConvNet as fixed feature extractor” 
fromhttps://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html and you can ignore “finetuning theConvNet”. Test this code out to see if it runs properly in your environment after eliminating code blocks thatyou do not need.

In [680]:

##                            https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
## this code is taken from link rovided in assignment
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir ="C:/Users/alnea/OneDrive/Desktop/hymenoptera_data"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# #Write a function that outputs ResNet18 features for a given input image. 
Extract features for training images(in image_datasets['train']). You should get an Nx512 dimensional array.

In [784]:
path_img='C:/Users/alnea/OneDrive/Desktop/hymenoptera_data'
# Define function to get features, shape of features
def resnet_18_feature_extraction(path):
        ## defining model
        model_resnet18 = models.resnet18(pretrained=True)

        ## Obtaining number of feature from last connected layer
        num_ftrs = model_resnet18.fc.in_features 

        # replacing last connected layer as identity
        model_resnet18.fc = torch.nn.Identity()

        # Setting model to evaluation mode to get reliable outputs
        model_resnet18.eval()
        ## these lines taken from link mentioned in question
        transform = transforms.Compose([                      ##  https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize( mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])])

        # Load the training dataset
        dataset = datasets.ImageFolder(path, transform=transform)    ##https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

        # data loader                         https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)  ##https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader

        # Extracting features for each image in the training dataset
        features_extracted = []
        # disabling the gradient to reduce memory consumption
        with torch.no_grad():                               ##https://pytorch.org/docs/stable/generated/torch.no_grad.html
            for images,labels in dataloader:
                extraction= model_resnet18(images)
                features_extracted.append(extraction)

        # Concatenating tensor of features & converting to numpy array
        features_extracted = torch.cat(features_extracted, dim=0).numpy()   ##https://pytorch.org/docs/stable/generated/torch.cat.html
        return features_extracted


In [785]:
features=resnet_18_feature_extraction (path_img)
features.shape

(397, 512)

In [792]:
## Getting train and test data from the features extracted and target from image_dataset
x_train=features[0:244]
x_test=features[244:397]
y_train=np.array(image_datasets['train'].targets)
y_test=np.array(image_datasets['val'].targets)

In [793]:
y_train.shape

(244,)

## Compare L2 regularized logistic regression, RBF kernel SVM (do grid search on kernel width andregularization), and random forest (do grid search on max depth and number of trees). 
Test the final modelon test data and show the results -- accuracy and F1 score

# a) L2 regularized logistic regression

In [789]:
# Importing LogisticRegression from sklearn.linear_model
from sklearn.linear_model import LogisticRegression
LR_model= LogisticRegression()                               ## https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# Making a hyperparameter dictionary 
hyperparameter_grid={'C': [0.001,0.01,0.1,1,10]}
#Calling gridsearch cv for hyperparameter tuning
grid_search_cv = GridSearchCV(estimator=LR_model,param_grid=hyperparameter_grid,cv=5,scoring='f1')
#fitting gridsearch sv on train data
grid_search_cv.fit(x_train, y_train)
#printing best hyperparameters
print('best_hyperparameters:',grid_search_cv.best_params_)
#printing best f1 score in the CVs
print('best_f1_score:',grid_search_cv.best_score_)

# Predicting the classification on the basis of test data
y_pred=grid_search_cv.predict(x_test)

# Getting f1 score
f1=f1_score(y_test,y_pred)

# Getting accuracy
acc=accuracy_score(y_test,y_pred)

#printing F1 score & Accuracy
print('f1_score:',f1)
print('Accuracy:',acc)

best_hyperparameters: {'C': 0.001}
best_f1_score: 0.9628893818396923
f1_score: 0.9634146341463414
Accuracy: 0.9607843137254902


# b) RBF kernel SVM (do grid search on kernel width andregularization)

In [788]:
# Making a hyperparameter dictionary 
hyperparameter_grid = {'C': [0.001,0.01,0.1,1,10],'kernel': ['rbf'],'gamma': [0.01,0.1,1,10,100],'class_weight':['balanced']} 
#Calling gridsearch cv for hyperparameter tuning
grid_search_cv = GridSearchCV(estimator=SVC(),param_grid=hyperparameter_grid,cv=5,scoring='f1')

#fitting gridsearch sv on train data
grid_search_cv.fit(x_train, y_train)

#printing best hyperparameters
print('best_hyperparameters:',grid_search_cv.best_params_)
#printing best f1 score in the CVs
print('best_f1_score:',grid_search_cv.best_score_)

# Predicting the classification on the basis of test data
y_pred=grid_search_cv.predict(x_test)

# Getting f1 score
f1=f1_score(y_test,y_pred)

# Getting accuracy
acc=accuracy_score(y_test,y_pred)

#printing F1 score & Accuracy
print('f1_score:',f1)
print('Accuracy:',acc)

best_hyperparameters: {'C': 1, 'class_weight': 'balanced', 'gamma': 0.01, 'kernel': 'rbf'}
best_f1_score: 0.936118100524206
f1_score: 0.9506172839506173
Accuracy: 0.9477124183006536


# c) Random forest (do grid search on max depth and number of trees)

In [787]:
# Making a hyperparameter dictionary 
hyperparameter_grid={'max_features':['sqrt','log2'],'n_estimators':[100,150,200,250],'class_weight':['balanced']}   #https://scikit-learn.org/stable/modules/grid_search.html#grid-search
## assigning RandomForestClassifier() model(estimator)
model= RandomForestClassifier(random_state=0)
#Calling gridsearch cv for hyperparameter tuning
grid_search_cv=GridSearchCV(estimator=model,param_grid=hyperparameter_grid,cv=5,scoring='f1')
#fitting gridsearch cv on train data
grid_search_cv.fit(x_train, y_train)

#printing best hyperparameters
print('best_hyperparameters:',grid_search_cv.best_params_)
#printing best f1 score in the CVs
print('best_f1_score:',grid_search_cv.best_score_)

# Predicting the classification on the basis of test data
y_pred=grid_search_cv.predict(x_test)

# Getting f1 score
f1=f1_score(y_test,y_pred)

# Getting accuracy
acc=accuracy_score(y_test,y_pred)

#printing F1 score & Accuracy
print('f1_score:',f1)
print('Accuracy:',acc)

best_hyperparameters: {'class_weight': 'balanced', 'max_features': 'sqrt', 'n_estimators': 100}
best_f1_score: 0.9543608114276807
f1_score: 0.9512195121951219
Accuracy: 0.9477124183006536


# Comparison



### ----------------Logistic Regression-------RBF---------------------------------------Randomforest

###### F1_score-----0.9634146341463414--------------0.9506172839506173------------------------------0.9512195121951219              
###### Accuracy------0.9607843137254902-------------0.9477124183006536----------------------------- 0.9477124183006536

### All three models are classifing on test data well and almost close accuracy & f1 score, But Linear Regression is giving slightly higher accuracy and f1 score

# 12). Summarize your findings and write your references.

Findings
1) How to do exploratory data analysis by variance of features, corelations between features,null value imputation and elimination

2) The Model gives high score on train data but low performance on test data, it means model are not that much generalizable.

3) How the pretrained resnet18 model can be used for feature extraction

4) It is importanat to give equal weight to all classes otherwise we get scores influenced by imbalance.

5) The neural network takes more time for hyperparameter tuning almost around half an hour

6) It is necessary to convert last layer of resnet18 as identity to get features from it.

7) for the execution of code where grad is not required, no grad should be execute to avoid memory consumption

8) Gridsearch CV does multiple cross validation which helps to get hyperparameter to generalize the model.

9) In Multiclass classification using linear SVM i got same score for recursive feature elimination model and the feature eliminated manually.

10) The model were performing good on the extracted features from resnet18 feature extractor.




##References 
I have mentioned links next to code line in code block
1) Andrew ng coursera neural network videos

2) 